<a href="https://colab.research.google.com/github/qcutexu/Introduction-to-Python3/blob/main/Linearized_simple_pressure_wave_by_Yifan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import dolfin
    import mshr
else:
    try:
        import dolfin
        import mshr
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import dolfin
        import mshr

--2022-09-24 23:45:24--  https://fem-on-colab.github.io/releases/fenics-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.111.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2831 (2.8K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   2.76K  --.-KB/s    in 0s      

2022-09-24 23:45:24 (25.9 MB/s) - ‘/tmp/fenics-install.sh’ saved [2831/2831]

+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/5710736/releases/pybind11-install.sh
+ [[ https://github.com/fem-on-colab/fem-on-colab.github.io/raw/5710736/releases/pybind11-install.sh == http* ]]
+ PYBIND11_

In [2]:
from dolfin import *
import matplotlib.pyplot as plt
from ufl import nabla_div, max_value
import numpy as np
import mshr
import random
import sys

#define domain and mesh 
domain = mshr.Rectangle(Point(0,0), Point(6,1))
mesh = mshr.generate_mesh(domain, 40) # RectangleMesh(Point(0,0), Point(20, 1), 200,10, 'right')
mesh_s = mesh #mshr.generate_mesh(domain_s, 50)

#define boundaries 
class Top_wall(SubDomain):
    def inside(self, x, on_boundary):
        return (near(x[1], 1.0)) # or near(x[1], 0))

class Bottom_wall(SubDomain):
    def inside(self, x, on_boundary):
        return (near(x[1], 0.0))

class leftWall(SubDomain):
    def inside(self, x, on_boundary):
        return (near(x[0], 0.0))

class rightWall(SubDomain):
    def inside(self, x, on_boundary):
        return (near(x[0], 6.0))

#call boundaries
top = Top_wall()
bottom = Bottom_wall()
leftwall=leftWall()
rightwall=rightWall()

#mark boundaries
boundaries = MeshFunction('size_t', mesh, mesh.topology().dim()-1,0)
boundaries_s = MeshFunction('size_t', mesh_s, mesh_s.topology().dim()-1,0)
Top, Bottom, Inlet, Outlet = 1, 2, 3, 4

#fluid boundaries labels
top.mark(boundaries, Top)
bottom.mark(boundaries, Bottom)
leftwall.mark(boundaries, Inlet)
rightwall.mark(boundaries, Outlet)
ds = Measure('ds', domain=mesh, subdomain_data=boundaries)

#structure boundaries labels
top.mark(boundaries_s, Top)
bottom.mark(boundaries_s, Bottom)
leftwall.mark(boundaries_s, Inlet)
rightwall.mark(boundaries_s, Outlet)
ds_s = Measure('ds', domain=mesh_s, subdomain_data=boundaries_s)

#define parameters ------------------------
t = 0.0; T = 0.012;
num_steps=400; dt=T/num_steps;
force = Constant((0., 0.))
mu = Constant(1.); rho = 1.;
nu=mu/rho;
n = FacetNormal(mesh);
InletPre=0;
e_r = Constant((0.0, 1.0))


E, nu_s = 0.75e6, 0.5
rho_s = 1;
h = 1;
#h=0.1;
R=0.5;
#Cspring= h*E/(1-nu_s**2)*(1/(R**2));
#C1= h**3*E*nu_s/(6*R**2*(1-nu_s**2));
InletPre = Expression('20000/2*(1-cos((2*pi*t)/0.005))*(t<0.005)', degree=5, t=0)

V = VectorFunctionSpace(mesh, 'CG', 2) # velocity space
Q = FunctionSpace(mesh, 'CG', 1) # pressure space
u1_ = Function(V) #for storing velocity solution
p1_ = Function(Q) #for current pressure

V_s = FunctionSpace(mesh_s, 'CG', 2) # structure fixed domain function space
Q_s = FunctionSpace(mesh_s, 'CG', 1) # pressure space
eta1_ = Function(V_s)
eta1f_ = Function(V_s)
ueta1_ = Function(V_s)
v1_ = Function(V_s)
ueta2_ = Function(V_s)



#for passing values to fluid domain
ueta1_f = Function(V_s)


# for noise term
f_noise = Constant(1.)
f_noise_V = interpolate(f_noise,V_s)

#define mixed function space for fluid subproblem
P2 = VectorElement("CG", mesh.ufl_cell(), 2)
P1 = FiniteElement("CG", mesh.ufl_cell(), 1)
element = MixedElement([P2, P1])
X = FunctionSpace(mesh, element)

#define mixed functions to save fluid subproblem results (u,p)
xh = Function(X)
tentative = Function(X)
u0, p0 = split(tentative)

#define trial and test functions
(u, p) = TrialFunctions(X) #for velocity, pressure
(v, q) = TestFunctions(X)
eta =TrialFunction(V_s)
ueta =TrialFunction(V_s)
etat =TestFunction(V_s)
uetat =TestFunction(V_s)
vBM = TrialFunction(V_s)
vBMt = TestFunction(V_s)

#define boundary conditions for pressure and velocity
# this is for the pressure
bcp1 = DirichletBC(X.sub(1), InletPre, boundaries, Inlet)
bcp2 = DirichletBC(X.sub(1), Constant(0), boundaries, Outlet)
bcu1 = DirichletBC(X.sub(0).sub(0), 0, boundaries_s, Top)
bcu2 = DirichletBC(X.sub(0).sub(0), 0, boundaries_s, Bottom)
bcu3 = DirichletBC(X.sub(0).sub(1), 0, boundaries_s, Inlet)
bcu4 = DirichletBC(X.sub(0).sub(1), 0, boundaries_s, Outlet)

bc = [bcu1,bcu2,bcu3,bcu4,bcp1, bcp2]


#define boundary condition for structure subproblem
bcd1 = DirichletBC(V_s, (0.0), boundaries_s, Inlet)
bcd2 = DirichletBC(V_s, (0.0), boundaries_s, Outlet)

bcd= [bcd1, bcd2]

#define boundary condition for the (mesh) laplacian equation
bcl1 = DirichletBC(V_s, eta1_, boundaries_s, Top)
bcl2 = DirichletBC(V_s, 0.0, boundaries_s, Inlet)
bcl3 = DirichletBC(V_s, 0.0, boundaries_s, Outlet)
bcl4 = DirichletBC(V_s, eta1_, boundaries_s, Bottom)
bcl= [bcl1, bcl4, bcl2, bcl3]


#define weak formulation Navier-Stokes------------
# Define symmetric gradient
def epsilon(u):
    return sym(nabla_grad(u))

# Define stress tensor
def sigma(u, p):
    return 2*mu*epsilon(u) - p*Identity(len(u))


# define brownian motion
def dW(delta_t):
    """Sample a random number at each call."""
    return np.random.normal(loc=0.0, scale=np.sqrt(delta_t))




#Fluid system
#Franziska order version
'''F_ns = rho/dt*dot((u - u0), v)*dx \
    + inner(sigma(u, p), epsilon(v))*dx \
    + (div(u)*q)*dx\
    + (dot(p*n, v))*ds(Inlet) \
    + (dot(p*n, v))*ds(Outlet) \
    + rho_s*h/dt*dot(dot(u,e_r)-ueta1_f,dot(v,e_r))*ds(Top) \
    + rho_s*h/dt*dot(dot(u,e_r)-ueta1_f,dot(v,e_r))*ds(Bottom) \
    + dot(p*n,v)*ds(Top) - dot(dot(2*mu*epsilon(u),n),v)*ds(Top) \
    + dot(p*n,v)*ds(Bottom) - dot(dot(2*mu*epsilon(u),n),v)*ds(Bottom)
'''
#Yifan trial version
F_ns = rho/dt*dot((u - u0), v)*dx \
    + inner(epsilon(u), epsilon(v))*dx \
    - (div(v)*p)*dx\
    + (div(u)*q)*dx\
    + (dot(p*n, v))*ds(Inlet) \
    + (dot(p*n, v))*ds(Outlet) \
    + rho_s*h/dt*dot(dot(u,e_r)-ueta1_f,dot(v,e_r))*ds(Top) \
    + rho_s*h/dt*dot(dot(u,e_r)-ueta1_f,dot(v,e_r))*ds(Bottom) \
    + dot(p*n,v)*ds(Top) - dot(dot(2*mu*epsilon(u),n),v)*ds(Top) \
    + dot(p*n,v)*ds(Bottom) - dot(dot(2*mu*epsilon(u),n),v)*ds(Bottom)

a_f, L_f = lhs(F_ns), rhs(F_ns)



F_s = rho_s*h/dt*dot(ueta-v1_,uetat)*ds_s(Top) \
    + rho_s*h/dt*dot(ueta-v1_,uetat)*ds_s(Bottom) \
    + 1e-14*dot(ueta,uetat)*dx \
    + inner(Dx(eta1_, 0), Dx(uetat,0))*ds_s(Top) \
    + inner(Dx(eta1_, 0), Dx(uetat,0))*ds_s(Bottom) \
    + dt*inner(Dx(ueta, 0), Dx(uetat,0))*ds_s(Top) \
    + dt*inner(Dx(ueta, 0), Dx(uetat,0))*ds_s(Bottom)


a_s, L_s = lhs(F_s), rhs(F_s)


# Stochastic Forcing update in weak form
F_BM = dot(vBM-ueta1_,vBMt)*ds_s(Top) \
    + 1e-14*dot(vBM,vBMt)*dx \
    + dot(vBM-ueta1_,vBMt)*ds_s(Bottom) \
    - dot(dot(e_r,n),dot(f_noise_V,vBMt))*ds_s(Top) \
    - dot(dot(e_r,n),dot(f_noise_V,vBMt))*ds_s(Bottom)

a_BM, L_BM = lhs(F_BM), rhs(F_BM)    




F_lap =- inner(nabla_grad(eta), nabla_grad(etat))*dx 

a_lap, L_lap = lhs(F_lap), rhs(F_lap)



#create save file
name_of_self = sys.argv[0].split('/')[-1].split('.')[0]
directory_xdmf='results/'+name_of_self+'/'
xdmffile_velocity = XDMFFile(directory_xdmf+'velocity.xdmf') 
xdmffile_pressure = XDMFFile(directory_xdmf+'pressure.xdmf')
xdmffile_displace = XDMFFile(directory_xdmf+'displace.xdmf')
xdmffile_velocity.parameters["flush_output"] = True
xdmffile_pressure.parameters["flush_output"] = True
xdmffile_displace.parameters["flush_output"] = True

#Solve problem

for i in range(num_steps):
    
    #update to current velocity and pressure
    tentative = xh
    u0, p0 = split(tentative)
    #eta1f0_.assign(eta1f_)
     
    InletPre.t=t
        
    # step 1, get v^{n+1/3}    
    solve(a_s == L_s, ueta1_, bcd) 

    # update eta^{n+1/3}=eta1_=eta^{n+1}
    eta1_.vector().axpy(dt,ueta1_.vector())
    #print(eta1_.vector().get_local()) # print out to check if values look reasonable

    # step 2, get v^{n+2/3} = ueta1_= ueta1_f
    
    # inc = dW(dt)
    # inc_const = Constant(inc)
    #inc_fcn = interpolate(inc_const,V_s)
    #f_noise_V.assign(inc_fcn)
    #solve(a_BM==L_BM,ueta2_)
    #ueta1_f.vector()[:]=ueta2_.vector()
    
    
    #    ueta1_.vector().axpy(inc,f_noise_V.vector()) # adding noise

    ueta1_f.vector()[:]=ueta1_.vector()
    


    solve(a_lap == L_lap, eta1f_, bcl)
   
    

    # step 3: get u^{n+1}, v^{n+1}
    solve(a_f == L_f, xh, bc)
    u1_.assign(xh.sub(0, deepcopy=True))
    p1_.assign(xh.sub(1, deepcopy=True))
    v1_.assign(xh.sub(0).sub(1,deepcopy=True))
 
    t += dt
    print('Current time = '+str(t))
    xdmffile_velocity.write(u1_, t); xdmffile_pressure.write(p1_, t)    
    xdmffile_displace.write(eta1f_,t);    
   
 

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_f4c31f6e8b0e7ddceb14ce599aa08d063c1bdf82

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00622511 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.548862 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representatio

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_6ab56968c6ffa883272fd990bd40fad8bf858cca

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00452733 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.0201409 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representati

Calling FFC just-in-time (JIT) compiler, this may take some time.


...

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_0bb6ea8295a9d49109447a98e18601bb13d2b05f

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00480652 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.0214424 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representati

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_858dc638dcda094edb4840bedc57cba6b0d209c8

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:UFL:Adjusting missing element cell to triangle.
INFO:UFL:Adjusting missing element cell to triangle.
INFO:UFL:Adjusting missing element cell to triangle.
INFO:UFL:Adjusting missing element cell to triangle.
INFO:FFC:  
INFO:FFC:  Geometric dimension:                 2
  Number of cell subdomains:           0
  Number of exterior_facet subdomains: 5
  Rank:                                2
  Arguments:                           '(v_0, v_1)'
  Number of coefficients:              2
  Coefficients:                        '[f_15, f_16]'
  Unique elements:                     'Mixed<Vector<2 x CG2(?,?)>, CG1(?,?)>, R0(?,
                                       ?), Vector<2 x R0(?

Current time = 0.0001
Current time = 0.0002
Current time = 0.00030000000000000003
Current time = 0.0004
Current time = 0.0005
Current time = 0.0006000000000000001
Current time = 0.0007000000000000001
Current time = 0.0008000000000000001
Current time = 0.0009000000000000002
Current time = 0.0010000000000000002
Current time = 0.0011000000000000003
Current time = 0.0012000000000000003
Current time = 0.0013000000000000004
Current time = 0.0014000000000000004
Current time = 0.0015000000000000005
Current time = 0.0016000000000000005
Current time = 0.0017000000000000006
Current time = 0.0018000000000000006
Current time = 0.0019000000000000006
Current time = 0.0020000000000000005
Current time = 0.0021000000000000003
Current time = 0.0022
Current time = 0.0023
Current time = 0.0024
Current time = 0.0024999999999999996
Current time = 0.0025999999999999994
Current time = 0.0026999999999999993
Current time = 0.002799999999999999
Current time = 0.002899999999999999
Current time = 0.0029999999999999

In [3]:
cd results/ipykernel_launcher/

/content/results/ipykernel_launcher


In [4]:
ls

displace.h5    pressure.h5    velocity.h5
displace.xdmf  pressure.xdmf  velocity.xdmf


In [5]:
from google.colab import files
files.download('displace.h5')
files.download('displace.xdmf')
files.download('pressure.h5')
files.download('velocity.h5')
files.download('pressure.xdmf')
files.download('velocity.xdmf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
cd ..

In [ ]:
cd ..

In [ ]:
rm -r results